In [11]:
# BIG DATA Project on Online Banking Analysis

#BY -:

# Prakash Singh (19CSU214)
# Ishit Sethi (19CSU136)
# Rahul


In [3]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *

# LOAN DATASET #

In [9]:
df = spark.read.csv("loan.csv", inferSchema = True, header = True)

In [9]:
df.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



In [10]:
df.show(5)

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450|      27675|            

In [6]:
len(df.columns)

15

In [7]:
df.count()

500

In [8]:
df.distinct().count()

500

In [13]:
#number of loans in each category
df.groupBy("Loan Category").count().orderBy("count", ascending = False).show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|         GOLD LOAN|   77|
|           HOUSING|   67|
|        AUTOMOBILE|   60|
|        TRAVELLING|   53|
|       RESTAURANTS|   41|
|COMPUTER SOFTWARES|   35|
|          SHOPPING|   35|
|          BUSINESS|   24|
|  EDUCATIONAL LOAN|   20|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
|           DINNING|   14|
|       ELECTRONICS|   14|
|       AGRICULTURE|   12|
|       BOOK STORES|    7|
|          BUILDING|    7|
+------------------+-----+



In [14]:
#number of people who have taken more than 1 lack loan
df.filter(df["Loan Amount"]>"1,00,000").count()

379

In [18]:
#number of people with income greater than 60000 rupees
df.filter(df["Income"]>"60000").count()

198

In [26]:
#number of people with 2 or more returned cheques and income less than 50000
df.filter((df[" Returned Cheque"]>"1") & (df["Income"]<"50000")).count()

137

In [27]:
#number of people with 2 or more returned cheques and are single
df.filter((df[" Returned Cheque"]>"1") & (df["Marital Status"]<"SINGLE")).count()

283

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+



In [117]:
rdd1 = df.filter(df["Marital Status"]=="SINGLE")
rdd2 = df.filter(df["Income"]>"50000")
result = rdd1.intersect(rdd2)
result.show()

+-----------+---+------+--------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|          Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|   Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+--------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|    IB14326| 37|  MALE|              DOCTOR|        SINGLE|          2| 56856|      23678|            7|      TRAVELLING|    588,690|      8|       87171|               4|                 3|
|    IB14655| 52|FEMALE|        BANK MANAGER|        SINGLE|          3| 68918|      30676|            5|      AUTOMOBILE|  1,265,945|      5|       16997|               5|                 4|
|    IB14489| 24|  MALE|             TEA

In [6]:
#number of people with expenditure over 50000 a month 
df.filter((df["Expenditure"]>"50000")).show()

+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|     Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14158| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 80000|      62541|            2|   AUTOMOBILE| 20,45,789 |      1|       16599|               2|                 3|
|    IB14176| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 80000|      62541|            2|      HOUSING| 20,45,789 |      1|       16599|               2|                 3|
|    IB14204| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 81000|      

# CREDIT CARD DATASET #

In [8]:
dfc = spark.read.csv("credit card.csv", inferSchema = True, header = True)

In [15]:
dfc.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [17]:
len(dfc.columns)

13

In [16]:
dfc.count()

10000

In [18]:
dfc.distinct().count()

10000

In [11]:
dfc.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [13]:
#number of members who are elgible for credit card
dfc.filter(dfc["CreditScore"]>700).count()

3116

In [19]:
#number of members who are  elgible and active in the bank
dfc.filter((dfc["IsActiveMember"]==1) & (dfc["CreditScore"]>700)).count()

1637

In [21]:
#credit card users in Spain 
dfc.filter(dfc["Geography"]=="Spain").show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        6|  15574012|      Chu|        645|    Spain|  Male| 44|     8|113755.78|            2|             0|      149756.71|     1|
|       12|  15737173|  Andrews|        497|    Spain|  Male| 24|     3|      0.0|            2|             0|       76390.01|     0|
|       15|  15600882|    Scott|        635|    Spain|F

In [23]:
dfc.filter((dfc["EstimatedSalary"]>100000) & (dfc["Exited"]==1)).count()

1044

In [70]:
#Eligibilty for people less than 600 credit score and is an active member

rdd3 = dfc.filter(dfc["CreditScore"]<"600")
rdd4 = dfc.filter(dfc["IsActiveMember"]=="1")
result = rdd3.intersect(rdd4)
result.show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|      892|  15780140|  Bellucci|        435|  Germany|  Male| 32|     2| 57017.06|            2|             1|        5907.11|     0|
|     1541|  15605950|Onwuamaeze|        530|  Germany|  Male| 23|     1|137060.88|            2|             1|      165227.23|     0|
|     1828|  15740104|      Tuan|        425|    Spain|Female| 22|     7|169649.73|            2|             1|       136365.0|     1|
|     2600|  15704844|    Hsiung|        550|    Spain|  Male| 62|     7| 80927.56|            1|             1|       64490.67|     0|
|     3468|  15771509|     Hirst|        538|  G

In [35]:
dfc.filter((dfc["EstimatedSalary"]<100000) & (dfc["NumOfProducts"]>1)).count()

2432

In [73]:
#Count of inactive members living in spain

cc1 = dfc.filter(dfc["Geography"]=="Spain")
cc2 = dfc.filter(dfc["IsActiveMember"]=="0")
result = cc1.union(cc2)
result.count()

7326

In [84]:
#Displaying France Female Users

rdd1 = dfc.filter(dfc["Geography"]=="France")
rdd2 = dfc.filter(dfc["Gender"]=="Male")
result = rdd1.subtract(rdd2)
result.show()


+---------+----------+-------------------+-----------+---------+------+---+------+--------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|            Surname|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------------------+-----------+---------+------+---+------+--------+-------------+--------------+---------------+------+
|      612|  15603203|           Avdeyeva|        650|   France|Female| 27|     6|     0.0|            2|             0|        1002.39|     0|
|     2758|  15566708|            Chidalu|        444|   France|Female| 45|     4|     0.0|            2|             0|       161653.5|     1|
|     3256|  15671387|    Fetherstonhaugh|        507|   France|Female| 29|     4|89349.47|            2|             0|      180626.68|     0|
|     3373|  15643967|            Chineze|        652|   France|Female| 37|     4|92208.54|            1|             1|       197699.8|

In [87]:
#Displaying pair wise frequency based on Gender and Geography

dfc.crosstab("Gender", "Geography").show()

+----------------+------+-------+-----+
|Gender_Geography|France|Germany|Spain|
+----------------+------+-------+-----+
|            Male|  2753|   1316| 1388|
|          Female|  2261|   1193| 1089|
+----------------+------+-------+-----+



In [90]:
#Removing dublicate Users with same surname

dfc.dropDuplicates(["Surname"]).show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|     3272|  15708791|     Abazu|        584|    Spain|  Male| 32|     9| 85534.83|            1|             0|      169137.24|     0|
|      842|  15737792|     Abbie|        818|   France|Female| 31|     1|186796.37|            1|             0|      178252.63|     0|
|     2538|  15723706|    Abbott|        573|   France|Female| 33|     0| 90124.64|            1|             0|      137476.71|     0|
|     8485|  15601012|  Abdullah|        802|   France|Female| 60|     3| 92887.06|            1|             0|       39473.63|     1|
|     6085|  15619494|   Abdulov|        562|  G

# TRANSACTION DATASET #

In [4]:
txn = spark.read.csv("txn.csv", inferSchema=True, header =True)

In [76]:
txn.printSchema()

root
 |-- Account No: string (nullable = true)
 |-- TRANSACTION DETAILS: string (nullable = true)
 |-- VALUE DATE: string (nullable = true)
 |--  WITHDRAWAL AMT : double (nullable = true)
 |--  DEPOSIT AMT : double (nullable = true)
 |-- BALANCE AMT: double (nullable = true)



In [20]:
#COUNT OF TRANSACTION ON EVERY ACCOUNT
txn.groupBy("Account No").count().show()

+-------------+-----+
|   Account No|count|
+-------------+-----+
|409000438611'| 4588|
|     1196711'|10536|
|     1196428'|48779|
|409000493210'| 6014|
|409000611074'| 1093|
|409000425051'|  802|
|409000405747'|   51|
|409000493201'| 1044|
|409000438620'|13454|
|409000362497'|29840|
+-------------+-----+



In [26]:
#Maximum withdrawal amount
txn.groupBy("Account No").max(" WITHDRAWAL AMT ").orderBy("max( WITHDRAWAL AMT )", ascending = False).show()

+-------------+---------------------+
|   Account No|max( WITHDRAWAL AMT )|
+-------------+---------------------+
|     1196711'|        4.594475464E8|
|409000438620'|                4.0E8|
|409000425051'|               3.54E8|
|409000438611'|                2.4E8|
|409000405747'|                1.7E8|
|     1196428'|                1.5E8|
|409000362497'|        1.413662392E8|
|409000493210'|                1.5E7|
|409000493201'|            2500000.0|
|409000611074'|             912000.0|
+-------------+---------------------+



In [24]:
#MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT
txn.groupBy("Account No").min(" WITHDRAWAL AMT ").orderBy("min( WITHDRAWAL AMT )").show()

+-------------+---------------------+
|   Account No|min( WITHDRAWAL AMT )|
+-------------+---------------------+
|409000493210'|                 0.01|
|409000438611'|                  0.2|
|     1196711'|                 0.25|
|     1196428'|                 0.25|
|409000438620'|                 0.34|
|409000362497'|                 0.97|
|409000425051'|                 1.25|
|409000493201'|                  2.1|
|409000405747'|                 21.0|
|409000611074'|                120.0|
+-------------+---------------------+



In [28]:
#MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT
txn.groupBy("Account No").max(" DEPOSIT AMT ").orderBy("max( DEPOSIT AMT )", ascending = False).show()

+-------------+------------------+
|   Account No|max( DEPOSIT AMT )|
+-------------+------------------+
|409000438620'|           5.448E8|
|     1196711'|             5.0E8|
|     1196428'|     2.119594422E8|
|409000405747'|           2.021E8|
|409000362497'|             2.0E8|
|409000438611'|          1.7025E8|
|409000425051'|             1.5E7|
|409000493210'|             1.5E7|
|409000611074'|         3000000.0|
|409000493201'|         1000000.0|
+-------------+------------------+



In [29]:
#MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT
txn.groupBy("Account No").min(" DEPOSIT AMT ").orderBy("min( DEPOSIT AMT )").show()

+-------------+------------------+
|   Account No|min( DEPOSIT AMT )|
+-------------+------------------+
|409000493210'|              0.01|
|409000438611'|              0.03|
|409000362497'|              0.03|
|409000438620'|              0.07|
|409000493201'|               0.9|
|     1196428'|               1.0|
|409000425051'|               1.0|
|     1196711'|              1.01|
|409000405747'|             500.0|
|409000611074'|            1320.0|
+-------------+------------------+



In [22]:
#sum of balance in every bank account
txn.groupBy("Account No").sum("BALANCE AMT").show()

+-------------+--------------------+
|   Account No|    sum(BALANCE AMT)|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
|409000362497'| -5.2860004792808E13|
+-------------+--------------------+



In [32]:
#Number of transaction on each date
txn.groupBy("VALUE DATE").count().orderBy("count", ascending = False).show()

+----------+-----+
|VALUE DATE|count|
+----------+-----+
| 27-Jul-17|  567|
| 13-Aug-18|  463|
|  8-Nov-17|  402|
|  7-Oct-17|  382|
| 10-Jul-18|  374|
| 12-Dec-17|  367|
| 12-Sep-18|  365|
|  9-Aug-18|  360|
| 19-Sep-17|  358|
| 16-Mar-17|  353|
| 10-Sep-18|  344|
| 14-Jul-17|  333|
|  7-Mar-18|  319|
| 11-Oct-18|  303|
| 22-Aug-17|  301|
|  9-Jan-18|  299|
|  9-Oct-18|  297|
| 20-Apr-18|  296|
|  9-Jul-18|  292|
|  7-Apr-18|  291|
+----------+-----+
only showing top 20 rows



In [33]:
#List of customers with withdrawal amount more than 1 lakh
txn.select("Account No","TRANSACTION DETAILS"," WITHDRAWAL AMT ").filter(txn[" WITHDRAWAL AMT "]>100000).show()

+-------------+--------------------+----------------+
|   Account No| TRANSACTION DETAILS| WITHDRAWAL AMT |
+-------------+--------------------+----------------+
|409000611074'|INDO GIBL Indiafo...|        133900.0|
|409000611074'|INDO GIBL Indiafo...|        195800.0|
|409000611074'|INDO GIBL Indiafo...|        143800.0|
|409000611074'|INDO GIBL Indiafo...|        331650.0|
|409000611074'|INDO GIBL Indiafo...|        129000.0|
|409000611074'|INDO GIBL Indiafo...|        230013.0|
|409000611074'|INDO GIBL Indiafo...|        367900.0|
|409000611074'|INDO GIBL Indiafo...|        108000.0|
|409000611074'|INDO GIBL Indiafo...|        141000.0|
|409000611074'|INDO GIBL Indiafo...|        206000.0|
|409000611074'|INDO GIBL Indiafo...|        242300.0|
|409000611074'|INDO GIBL Indiafo...|        113250.0|
|409000611074'|INDO GIBL Indiafo...|        206900.0|
|409000611074'|INDO GIBL Indiafo...|        276000.0|
|409000611074'|INDO GIBL Indiafo...|        171000.0|
|409000611074'|INDO GIBL Ind

In [101]:
#Displaying only those records having null values corresponding to withdrawal amount

txn.dropna(subset=[' WITHDRAWAL AMT ']).show()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        133900.0|         null|  8366100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         18000.0|         null|  8348100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|          5000.0|         null|  8343100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        195800.0|         null|  8147300.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         81600.0|         null|  8065700.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         41800.0|         null|  8023900.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         98500.0|         null|  7925400.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        143800.0|         null|  7781600.0|

In [10]:
#Displaying Single users with income more than 50,000

rdd1 = df.filter(df["Marital Status"]=="SINGLE")
rdd2 = df.filter(df["Income"]>"50000")
result = rdd1.intersect(rdd2)
result.show()

+-----------+---+------+--------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|          Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|   Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+--------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|    IB14326| 37|  MALE|              DOCTOR|        SINGLE|          2| 56856|      23678|            7|      TRAVELLING|    588,690|      8|       87171|               4|                 3|
|    IB14655| 52|FEMALE|        BANK MANAGER|        SINGLE|          3| 68918|      30676|            5|      AUTOMOBILE|  1,265,945|      5|       16997|               5|                 4|
|    IB14489| 24|  MALE|             TEA

In [106]:
#Displaying only transaction related informations using colRegex 

txn.select(txn.colRegex("`^.*AMT.*`")).show()

+----------------+-------------+-----------+
| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+----------------+-------------+-----------+
|            null|    1000000.0|  1000000.0|
|            null|    1000000.0|  2000000.0|
|            null|     500000.0|  2500000.0|
|            null|    3000000.0|  5500000.0|
|            null|     500000.0|  6000000.0|
|            null|     500000.0|  6500000.0|
|            null|     500000.0|  7000000.0|
|            null|     500000.0|  7500000.0|
|            null|     500000.0|  8000000.0|
|            null|     500000.0|  8500000.0|
|        133900.0|         null|  8366100.0|
|         18000.0|         null|  8348100.0|
|          5000.0|         null|  8343100.0|
|        195800.0|         null|  8147300.0|
|         81600.0|         null|  8065700.0|
|         41800.0|         null|  8023900.0|
|         98500.0|         null|  7925400.0|
|        143800.0|         null|  7781600.0|
|        331650.0|         null|  7449950.0|
|        1

In [108]:
#Showing summary of tansaction dataset

txn.summary().show()

+-------+-------------+--------------------+----------+--------------------+-----------------+--------------------+
|summary|   Account No| TRANSACTION DETAILS|VALUE DATE|     WITHDRAWAL AMT |     DEPOSIT AMT |         BALANCE AMT|
+-------+-------------+--------------------+----------+--------------------+-----------------+--------------------+
|  count|       116201|              113702|    116201|               53549|            62652|              116201|
|   mean|         null|3.675022545399418E15|      null|   4489189.943506325|3806585.828440277|-1.40485204095910...|
| stddev|         null|1.493425698537586...|      null|1.0848504204717927E7|8683093.407864038| 5.348201823458226E8|
|    min|     1196428'|(SR1239979079) RE...|  1-Apr-17|                0.01|             0.01|      -2.045201142E9|
|    25%|         null|             6.33E11|      null|              3000.0|         98962.35|       -1.69038345E9|
|    50%|         null|             7.21E11|      null|             4705

In [109]:
#Displaying data with renamed column name

txn.withColumnRenamed("Account No", "ACCOUNT NUMBER").show()

+--------------+--------------------+----------+----------------+-------------+-----------+
|ACCOUNT NUMBER| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+--------------+--------------------+----------+----------------+-------------+-----------+
| 409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            null|    1000000.0|  1000000.0|
| 409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            null|    1000000.0|  2000000.0|
| 409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            null|     500000.0|  2500000.0|
| 409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            null|    3000000.0|  5500000.0|
| 409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  6000000.0|
| 409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  6500000.0|
| 409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  7000000.0|
| 409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0| 

In [113]:
#Showing only first record of dataset

txn.first()

Row(Account No="409000611074'", TRANSACTION DETAILS='TRF FROM  Indiaforensic SERVICES', VALUE DATE='29-Jun-17',  WITHDRAWAL AMT =None,  DEPOSIT AMT =1000000.0, BALANCE AMT=1000000.0)

In [116]:
#Displaying summary of particular column field

txn.describe(" WITHDRAWAL AMT ")

DataFrame[summary: string,  WITHDRAWAL AMT : string]

In [6]:
#Replacing Null value with 0

txn.na.fill(0).show()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|             0.0|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|             0.0|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|             0.0|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|             0.0|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  7500000.0|